In [ ]:
import pandas as pd

df1= pd.read_csv('GlobalLandTemperaturesByCountry.csv')
df2 = pd.read_csv('AnnualSurfaceTemperatureChange.csv', encoding='latin1')

In [ ]:
df1

In [ ]:
df2

In [ ]:
unique_emissions = df2['Gas_Type'].unique()
print(unique_emissions)
unique_industries = df2['Industry'].unique()
print(unique_industries)

In [ ]:
df2 = df2.drop(['CTS_Code', 'CTS_Name', 'ISO3', 'Indicator'], axis=1)

# Display the resulting DataFrame
df2


In [ ]:
# Melt the DataFrame to convert it from wide to long format
df2_melted = df2.melt(id_vars=['Country', 'Unit', 'Industry', 'Gas_Type'], var_name='year', value_name='Million metric tons of CO2 equivalent')

# Extract the year from the 'year' column and convert it to integer
df2_melted['year'] = df2_melted['year'].str.extract('(\d+)').astype(int)

# Get rid of years after 2013
df2_filtered = df2_melted[df2_melted['year'] <= 2013]
df2 = df2_filtered

df2

In [7]:
df1['dt'] = pd.to_datetime(df1['dt'])

# Filter rows with years 1970 and after
df1 = df1[df1['dt'].dt.year >= 1970]

# Drop rows with Nans
df1 = df1.dropna()

# Calculate average temperature, and add a new column
df1['Year'] = df1['dt'].dt.year
df1['AverageYearlyTemperature'] = df1.groupby(['Country', 'Year'])['AverageTemperature'].transform('mean')
df1['Month'] = df1['dt'].dt.month

# Find the index of the coldest month, and add a new column
df1['ColdestMonth'] = df1.groupby(['Country', 'Year'])['AverageTemperature'].transform(lambda x: x.idxmin())
df1['ColdestMonth'] = df1['ColdestMonth'].apply(lambda x: df1.loc[x, 'Month'])
df1['ColdestMonthIndex'] = df1.groupby(['Country', 'Year'])['AverageTemperature'].transform(lambda x: x.idxmin())

# Extract temperature in the coldest month
df1['ColdestMonthTemperature'] = df1.apply(lambda row: df1.loc[row['ColdestMonthIndex'], 'AverageTemperature'], axis=1)

# Find the index of the hottest month, and add a new column
df1['HottestMonthIndex'] = df1.groupby(['Country', 'Year'])['AverageTemperature'].transform(lambda x: x.idxmax())

# Extract month and temperature in the hottest month
df1['HottestMonth'] = df1.apply(lambda row: df1.loc[row['HottestMonthIndex'], 'Month'], axis=1)
df1['HottestMonthTemperature'] = df1.apply(lambda row: df1.loc[row['HottestMonthIndex'], 'AverageTemperature'], axis=1)

# Drop unnecessary columns
df1 = df1.drop(['HottestMonthIndex', 'Month', 'ColdestMonthIndex', 'AverageTemperature', 'AverageTemperatureUncertainty'], axis=1)

# Remove duplicates to keep only one row per year for each country
df1 = df1.drop_duplicates(subset=['Country', 'Year'])

# Reset index
df1 = df1.reset_index(drop=True)

print(df1)

              dt   Country  Year  AverageYearlyTemperature  ColdestMonth  \
0     1970-01-01     Åland  1970                  4.442833             2   
1     1971-01-01     Åland  1971                  5.623750             3   
2     1972-01-01     Åland  1972                  6.358333             1   
3     1973-01-01     Åland  1973                  6.198083            12   
4     1974-01-01     Åland  1974                  6.766917             3   
...          ...       ...   ...                       ...           ...   
10643 2009-01-01  Zimbabwe  2009                 21.377250             7   
10644 2010-01-01  Zimbabwe  2010                 21.986250             6   
10645 2011-01-01  Zimbabwe  2011                 21.602417             7   
10646 2012-01-01  Zimbabwe  2012                 21.521333             7   
10647 2013-01-01  Zimbabwe  2013                 20.710750             7   

       ColdestMonthTemperature  HottestMonth  HottestMonthTemperature  
0              

In [8]:
month_names = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

# Map month numbers to their names for coldest and hottest months
df1['ColdestMonth'] = df1['ColdestMonth'].map(month_names)
df1['HottestMonth'] = df1['HottestMonth'].map(month_names)

# Calculate the difference in average yearly temperature from the previous year
df1['YearlyTemperatureChange'] = df1.groupby('Country')['AverageYearlyTemperature'].diff()
#Replace Nans with zero
df1.fillna(0, inplace=True)
df1

,dt,Country,Year,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange
0,1970-01-01,Åland,1970,4.442833,February,-10.835,August,15.416,0.000000
1,1971-01-01,Åland,1971,5.623750,March,-3.457,July,15.677,1.180917
2,1972-01-01,Åland,1972,6.358333,January,-3.942,July,18.196,0.734583
3,1973-01-01,Åland,1973,6.198083,December,-2.136,July,18.634,-0.160250
4,1974-01-01,Åland,1974,6.766917,March,0.077,August,15.067,0.568833
...,...,...,...,...,...,...,...,...,...
10643,2009-01-01,Zimbabwe,2009,21.377250,July,15.234,December,24.731,-0.168333
10644,2010-01-01,Zimbabwe,2010,21.986250,June,16.377,October,25.943,0.609000
10645,2011-01-01,Zimbabwe,2011,21.602417,July,15.803,November,25.521,-0.383833
10646,2012-01-01,Zimbabwe,2012,21.521333,July,16.048,November,24.606,-0.081083


In [9]:
merged_df = pd.merge(df1, df2, on='Country')
merged_df

,dt,Country,Year,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,Unit,Industry,Gas_Type,year,Million metric tons of CO2 equivalent
0,1970-01-01,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Carbon dioxide,1970,1.228059
1,1970-01-01,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Greenhouse gas,1970,324.318696
2,1970-01-01,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Methane,1970,220.225910
3,1970-01-01,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Nitrous oxide,1970,102.864726
4,1970-01-01,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Buildings and other Sectors,Carbon dioxide,1970,35.376012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23013227,2013-01-01,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Carbon dioxide,2013,24.075733
23013228,2013-01-01,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Fluorinated gases,2013,0.260835
23013229,2013-01-01,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Greenhouse gas,2013,43.101272
23013230,2013-01-01,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Methane,2013,13.307377


In [10]:
merged_df = merged_df.drop(['dt'], axis=1)
merged_df = merged_df.rename(columns={'Unit': 'Emissions Unit', 'Million metric tons of CO2 equivalent': 'Emissions'})
merged_df

,Country,Year,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,Emissions Unit,Industry,Gas_Type,year,Emissions
0,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Carbon dioxide,1970,1.228059
1,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Greenhouse gas,1970,324.318696
2,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Methane,1970,220.225910
3,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Nitrous oxide,1970,102.864726
4,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Buildings and other Sectors,Carbon dioxide,1970,35.376012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23013227,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Carbon dioxide,2013,24.075733
23013228,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Fluorinated gases,2013,0.260835
23013229,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Greenhouse gas,2013,43.101272
23013230,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Methane,2013,13.307377


In [11]:
merged_df = merged_df[merged_df['Gas_Type'] != 'Not Applicable']

# Reset index after filtering
merged_df.reset_index(drop=True, inplace=True)

merged_df

,Country,Year,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,Emissions Unit,Industry,Gas_Type,year,Emissions
0,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Carbon dioxide,1970,1.228059
1,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Greenhouse gas,1970,324.318696
2,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Methane,1970,220.225910
3,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Nitrous oxide,1970,102.864726
4,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Buildings and other Sectors,Carbon dioxide,1970,35.376012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20109227,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Carbon dioxide,2013,24.075733
20109228,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Fluorinated gases,2013,0.260835
20109229,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Greenhouse gas,2013,43.101272
20109230,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Methane,2013,13.307377


In [12]:
grouped = merged_df.groupby(['Country', 'year', 'Gas_Type'])['Emissions'].sum().unstack(fill_value=0)

# Rename columns for clarity
grouped.columns = ['CO2_Emissions', 'GreenhouseGas_Emissions', 'Methane_Emissions', 'NitrousOxide_Emissions', 'FluorinatedGases_Emissions']

# Merge the grouped data back into the original dataframe
merged_df = pd.merge(merged_df, grouped, how='left', on=['Country', 'year'])

# Display the resulting dataframe
merged_df

,Country,Year,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,Emissions Unit,Industry,Gas_Type,year,Emissions,CO2_Emissions,GreenhouseGas_Emissions,Methane_Emissions,NitrousOxide_Emissions,FluorinatedGases_Emissions
0,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Carbon dioxide,1970,1.228059,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
1,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Greenhouse gas,1970,324.318696,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
2,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Methane,1970,220.225910,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
3,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,Nitrous oxide,1970,102.864726,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
4,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Buildings and other Sectors,Carbon dioxide,1970,35.376012,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20109227,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Carbon dioxide,2013,24.075733,3240.374734,45.906950,5890.533520,1864.131589,740.120247
20109228,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Fluorinated gases,2013,0.260835,3240.374734,45.906950,5890.533520,1864.131589,740.120247
20109229,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Greenhouse gas,2013,43.101272,3240.374734,45.906950,5890.533520,1864.131589,740.120247
20109230,Zimbabwe,2013,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,Methane,2013,13.307377,3240.374734,45.906950,5890.533520,1864.131589,740.120247


In [13]:
merged_df = merged_df.drop(columns=['Gas_Type'])
merged_df = merged_df.drop(columns=['Year'])

merged_df = merged_df.drop_duplicates()

merged_df

,Country,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,Emissions Unit,Industry,year,Emissions,CO2_Emissions,GreenhouseGas_Emissions,Methane_Emissions,NitrousOxide_Emissions,FluorinatedGases_Emissions
0,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,1970,1.228059,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
1,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,1970,324.318696,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
2,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,1970,220.225910,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
3,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Agriculture,1970,102.864726,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
4,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,Buildings and other Sectors,1970,35.376012,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20109226,Zimbabwe,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,2013,5.346914,3240.374734,45.906950,5890.533520,1864.131589,740.120247
20109227,Zimbabwe,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,2013,24.075733,3240.374734,45.906950,5890.533520,1864.131589,740.120247
20109229,Zimbabwe,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,2013,43.101272,3240.374734,45.906950,5890.533520,1864.131589,740.120247
20109230,Zimbabwe,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,Not Applicable,2013,13.307377,3240.374734,45.906950,5890.533520,1864.131589,740.120247


In [14]:
merged_df = merged_df.drop(columns=['Emissions'])
merged_df = merged_df.drop(columns=['Industry'])


merged_df = merged_df.drop_duplicates()

merged_df = merged_df.reset_index(drop=True)

merged_df

,Country,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,Emissions Unit,year,CO2_Emissions,GreenhouseGas_Emissions,Methane_Emissions,NitrousOxide_Emissions,FluorinatedGases_Emissions
0,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,1970,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
1,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,1971,65698.449942,230.533470,167785.877827,84608.622741,17248.271662
2,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,1972,71123.001404,256.200155,176209.783378,87357.595343,17472.986474
3,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,1973,80463.562871,292.605500,189116.514777,90787.750991,17572.595426
4,Africa,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,1974,84212.720530,328.357094,193014.219266,90651.556696,17821.584994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265227,Zimbabwe,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,2009,2385.117265,43.824745,4726.987794,1673.450995,624.594788
265228,Zimbabwe,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,2010,2738.667402,43.529480,5218.071099,1753.776572,682.097645
265229,Zimbabwe,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,2011,3090.699304,43.806655,5651.261942,1806.427599,710.328385
265230,Zimbabwe,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,2012,3188.778694,44.685379,5800.089127,1842.329773,724.295280


In [15]:
# Gets a list of column names
columns = list(merged_df.columns)

# Determine the column name you want to move and the target column name
column_to_move = 'year'
target_column = 'Country'

columns.remove(column_to_move)

# Find the index position of the target column
target_index = columns.index(target_column)

# Insert the removed column name after the target column name
columns.insert(target_index + 1, column_to_move)

# Rearranges the columns of the data frame according to the updated list of column names
merged_df = merged_df[columns]


merged_df = merged_df.rename(columns={'year': 'Year'})

merged_df


,Country,Year,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,Emissions Unit,CO2_Emissions,GreenhouseGas_Emissions,Methane_Emissions,NitrousOxide_Emissions,FluorinatedGases_Emissions
0,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,64870.726312,184.519702,165907.310511,83831.162105,17020.902415
1,Africa,1971,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,65698.449942,230.533470,167785.877827,84608.622741,17248.271662
2,Africa,1972,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,71123.001404,256.200155,176209.783378,87357.595343,17472.986474
3,Africa,1973,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,80463.562871,292.605500,189116.514777,90787.750991,17572.595426
4,Africa,1974,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,84212.720530,328.357094,193014.219266,90651.556696,17821.584994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265227,Zimbabwe,2009,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,2385.117265,43.824745,4726.987794,1673.450995,624.594788
265228,Zimbabwe,2010,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,2738.667402,43.529480,5218.071099,1753.776572,682.097645
265229,Zimbabwe,2011,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,3090.699304,43.806655,5651.261942,1806.427599,710.328385
265230,Zimbabwe,2012,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,3188.778694,44.685379,5800.089127,1842.329773,724.295280


In [16]:

# Caculate the sum of these emissions
emissions_columns = ['CO2_Emissions', 'GreenhouseGas_Emissions', 'Methane_Emissions', 'NitrousOxide_Emissions', 'FluorinatedGases_Emissions']

merged_df['Total_emissions'] = merged_df[emissions_columns].sum(axis=1)

merged_df

,Country,Year,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,Emissions Unit,CO2_Emissions,GreenhouseGas_Emissions,Methane_Emissions,NitrousOxide_Emissions,FluorinatedGases_Emissions,Total_emissions
0,Africa,1970,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,64870.726312,184.519702,165907.310511,83831.162105,17020.902415,331814.621045
1,Africa,1971,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,65698.449942,230.533470,167785.877827,84608.622741,17248.271662,335571.755642
2,Africa,1972,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,71123.001404,256.200155,176209.783378,87357.595343,17472.986474,352419.566753
3,Africa,1973,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,80463.562871,292.605500,189116.514777,90787.750991,17572.595426,378233.029565
4,Africa,1974,24.321583,December,20.988,May,25.910,0.000000,Million metric tons of CO2 equivalent,84212.720530,328.357094,193014.219266,90651.556696,17821.584994,386028.438580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265227,Zimbabwe,2009,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,2385.117265,43.824745,4726.987794,1673.450995,624.594788,9453.975589
265228,Zimbabwe,2010,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,2738.667402,43.529480,5218.071099,1753.776572,682.097645,10436.142198
265229,Zimbabwe,2011,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,3090.699304,43.806655,5651.261942,1806.427599,710.328385,11302.523885
265230,Zimbabwe,2012,20.710750,July,17.000,February,24.075,-0.810583,Million metric tons of CO2 equivalent,3188.778694,44.685379,5800.089127,1842.329773,724.295280,11600.178253


In [17]:
merged_df = merged_df.drop(columns=['AverageYearlyTemperature'])
merged_df = merged_df.drop(columns=['ColdestMonth'])
merged_df = merged_df.drop(columns=['ColdestMonthTemperature'])
merged_df = merged_df.drop(columns=['HottestMonth'])
merged_df = merged_df.drop(columns=['HottestMonthTemperature'])
merged_df = merged_df.drop(columns=['YearlyTemperatureChange'])

merged_df = merged_df.merge(df1[['Country', 'Year', 'AverageYearlyTemperature', 'ColdestMonth', 'ColdestMonthTemperature', 'HottestMonth', 'HottestMonthTemperature', 'YearlyTemperatureChange']], on=['Country', 'Year'], how='left')

merged_df

,Country,Year,Emissions Unit,CO2_Emissions,GreenhouseGas_Emissions,Methane_Emissions,NitrousOxide_Emissions,FluorinatedGases_Emissions,Total_emissions,AverageYearlyTemperature,ColdestMonth,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange
0,Africa,1970,Million metric tons of CO2 equivalent,64870.726312,184.519702,165907.310511,83831.162105,17020.902415,331814.621045,24.321583,December,20.988,May,25.910,0.000000
1,Africa,1971,Million metric tons of CO2 equivalent,65698.449942,230.533470,167785.877827,84608.622741,17248.271662,335571.755642,23.933083,January,20.897,May,25.542,-0.388500
2,Africa,1972,Million metric tons of CO2 equivalent,71123.001404,256.200155,176209.783378,87357.595343,17472.986474,352419.566753,24.199083,January,20.998,September,26.230,0.266000
3,Africa,1973,Million metric tons of CO2 equivalent,80463.562871,292.605500,189116.514777,90787.750991,17572.595426,378233.029565,24.400083,December,20.976,May,26.209,0.201000
4,Africa,1974,Million metric tons of CO2 equivalent,84212.720530,328.357094,193014.219266,90651.556696,17821.584994,386028.438580,23.830417,January,20.502,June,25.670,-0.569667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265227,Zimbabwe,2009,Million metric tons of CO2 equivalent,2385.117265,43.824745,4726.987794,1673.450995,624.594788,9453.975589,21.377250,July,15.234,December,24.731,-0.168333
265228,Zimbabwe,2010,Million metric tons of CO2 equivalent,2738.667402,43.529480,5218.071099,1753.776572,682.097645,10436.142198,21.986250,June,16.377,October,25.943,0.609000
265229,Zimbabwe,2011,Million metric tons of CO2 equivalent,3090.699304,43.806655,5651.261942,1806.427599,710.328385,11302.523885,21.602417,July,15.803,November,25.521,-0.383833
265230,Zimbabwe,2012,Million metric tons of CO2 equivalent,3188.778694,44.685379,5800.089127,1842.329773,724.295280,11600.178253,21.521333,July,16.048,November,24.606,-0.081083


In [18]:
num_bins = 10
# Perform bucketizing for each emission type and total emissions
merged_df['CO2_Emissions_Bucket'] = pd.qcut(merged_df['CO2_Emissions'], q=num_bins, duplicates='drop')
merged_df['GreenhouseGas_Emissions_Bucket'] = pd.qcut(merged_df['GreenhouseGas_Emissions'], q=num_bins, duplicates='drop')
merged_df['Methane_Emissions_Bucket'] = pd.qcut(merged_df['Methane_Emissions'], q=num_bins, duplicates='drop')
merged_df['NitrousOxide_Emissions_Bucket'] = pd.qcut(merged_df['NitrousOxide_Emissions'], q=num_bins, duplicates='drop')
merged_df['FluorinatedGases_Emissions_Bucket'] = pd.qcut(merged_df['FluorinatedGases_Emissions'], q=num_bins, duplicates='drop')
merged_df['Total_emissions_Bucket'] = pd.qcut(merged_df['Total_emissions'], q=num_bins, duplicates='drop')

merged_df

,Country,Year,Emissions Unit,CO2_Emissions,GreenhouseGas_Emissions,Methane_Emissions,NitrousOxide_Emissions,FluorinatedGases_Emissions,Total_emissions,AverageYearlyTemperature,...,ColdestMonthTemperature,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,CO2_Emissions_Bucket,GreenhouseGas_Emissions_Bucket,Methane_Emissions_Bucket,NitrousOxide_Emissions_Bucket,FluorinatedGases_Emissions_Bucket,Total_emissions_Bucket
0,Africa,1970,Million metric tons of CO2 equivalent,64870.726312,184.519702,165907.310511,83831.162105,17020.902415,331814.621045,24.321583,...,20.988,May,25.910,0.000000,"(63267.757, 3408222.516]","(75.487, 233.043]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]"
1,Africa,1971,Million metric tons of CO2 equivalent,65698.449942,230.533470,167785.877827,84608.622741,17248.271662,335571.755642,23.933083,...,20.897,May,25.542,-0.388500,"(63267.757, 3408222.516]","(75.487, 233.043]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]"
2,Africa,1972,Million metric tons of CO2 equivalent,71123.001404,256.200155,176209.783378,87357.595343,17472.986474,352419.566753,24.199083,...,20.998,September,26.230,0.266000,"(63267.757, 3408222.516]","(233.043, 979.53]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]"
3,Africa,1973,Million metric tons of CO2 equivalent,80463.562871,292.605500,189116.514777,90787.750991,17572.595426,378233.029565,24.400083,...,20.976,May,26.209,0.201000,"(63267.757, 3408222.516]","(233.043, 979.53]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]"
4,Africa,1974,Million metric tons of CO2 equivalent,84212.720530,328.357094,193014.219266,90651.556696,17821.584994,386028.438580,23.830417,...,20.502,June,25.670,-0.569667,"(63267.757, 3408222.516]","(233.043, 979.53]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265227,Zimbabwe,2009,Million metric tons of CO2 equivalent,2385.117265,43.824745,4726.987794,1673.450995,624.594788,9453.975589,21.377250,...,15.234,December,24.731,-0.168333,"(1993.736, 4133.465]","(24.897, 75.487]","(4327.655, 7638.787]","(1221.238, 1953.222]","(596.912, 883.45]","(8655.31, 15277.575]"
265228,Zimbabwe,2010,Million metric tons of CO2 equivalent,2738.667402,43.529480,5218.071099,1753.776572,682.097645,10436.142198,21.986250,...,16.377,October,25.943,0.609000,"(1993.736, 4133.465]","(24.897, 75.487]","(4327.655, 7638.787]","(1221.238, 1953.222]","(596.912, 883.45]","(8655.31, 15277.575]"
265229,Zimbabwe,2011,Million metric tons of CO2 equivalent,3090.699304,43.806655,5651.261942,1806.427599,710.328385,11302.523885,21.602417,...,15.803,November,25.521,-0.383833,"(1993.736, 4133.465]","(24.897, 75.487]","(4327.655, 7638.787]","(1221.238, 1953.222]","(596.912, 883.45]","(8655.31, 15277.575]"
265230,Zimbabwe,2012,Million metric tons of CO2 equivalent,3188.778694,44.685379,5800.089127,1842.329773,724.295280,11600.178253,21.521333,...,16.048,November,24.606,-0.081083,"(1993.736, 4133.465]","(24.897, 75.487]","(4327.655, 7638.787]","(1221.238, 1953.222]","(596.912, 883.45]","(8655.31, 15277.575]"


In [19]:
##Bucketizing All emission types

CO2_Emissions_bin_counts = merged_df['CO2_Emissions_Bucket'].value_counts().sort_index()
print("Distribution of CO2 Emissions across different emission amounts (using an equal width binning):")
print(CO2_Emissions_bin_counts)
GreenhouseGas_bin_counts = merged_df['GreenhouseGas_Emissions_Bucket'].value_counts().sort_index()
print("Distribution of Greenhouse Gas Emissions across different emission amounts (using an equal width binning):")
print(GreenhouseGas_bin_counts)
Methane_Emissions_bin_counts = merged_df['Methane_Emissions_Bucket'].value_counts().sort_index()
print("Distribution of Methane Emissions across different emission amounts (using an equal width binning):")
print(Methane_Emissions_bin_counts)
NitrousOxide_Emissions_bin_counts = merged_df['NitrousOxide_Emissions_Bucket'].value_counts().sort_index()
print("Distribution of Nitrous Oxide Emissions across different emission amounts (using an equal width binning):")
print(NitrousOxide_Emissions_bin_counts)
FluorinatedGases_bin_counts = merged_df['FluorinatedGases_Emissions_Bucket'].value_counts().sort_index()
print("Distribution of Fluorinated Gases Emissions across different emission amounts (using an equal width binning):")
print(FluorinatedGases_bin_counts)
Total_emissions_bin_counts = merged_df['Total_emissions_Bucket'].value_counts().sort_index()
print("Distribution of Total Emissions across different emission amounts (using an equal width binning):")
print(Total_emissions_bin_counts)

Distribution of CO2 Emissions across different emission amounts (using an equal width binning):
CO2_Emissions_Bucket
(-2757.132, 32.481]         26532
(32.481, 163.169]           26532
(163.169, 402.696]          26532
(402.696, 1010.709]         26532
(1010.709, 1993.736]        26488
(1993.736, 4133.465]        26532
(4133.465, 7428.731]        26532
(7428.731, 15297.976]       26532
(15297.976, 63267.757]      26532
(63267.757, 3408222.516]    26488
Name: count, dtype: int64
Distribution of Greenhouse Gas Emissions across different emission amounts (using an equal width binning):
GreenhouseGas_Emissions_Bucket
(-0.001, 0.0807]        106128
(0.0807, 2.592]          26488
(2.592, 24.897]          26532
(24.897, 75.487]         26576
(75.487, 233.043]        26488
(233.043, 979.53]        26532
(979.53, 102996.572]     26488
Name: count, dtype: int64
Distribution of Methane Emissions across different emission amounts (using an equal width binning):
Methane_Emissions_Bucket
(-1473.8129

In [20]:
##Bucketizing of average yearly temperature change
merged_df['YearlyTemperatureChange_Bucket'] = pd.qcut(merged_df['YearlyTemperatureChange'], q=num_bins, duplicates='drop')
YearlyTemperatureChange_bin_counts = merged_df['YearlyTemperatureChange_Bucket'].value_counts().sort_index()
print("Distribution of Yearly Temperature Change across different Temperature Change amounts (using an equal width binning):")
print(YearlyTemperatureChange_bin_counts)

Distribution of Yearly Temperature Change across different Temperature Change amounts (using an equal width binning):
YearlyTemperatureChange_Bucket
(-3.1069999999999998, -0.581]    26532
(-0.581, -0.339]                 26532
(-0.339, -0.178]                 26532
(-0.178, -0.0617]                26532
(-0.0617, 0.0133]                26532
(0.0133, 0.113]                  26488
(0.113, 0.237]                   26532
(0.237, 0.382]                   26532
(0.382, 0.619]                   26532
(0.619, 3.404]                   26488
Name: count, dtype: int64


In [21]:
# Surrogate Key Generation
merged_df['Surrogate Keys'] = range(1,len(merged_df)+1)

merged_df = merged_df.reindex(columns=['Surrogate Keys'] + list([c for c in merged_df.columns if c!= 'Surrogate Keys']))

merged_df

,Surrogate Keys,Country,Year,Emissions Unit,CO2_Emissions,GreenhouseGas_Emissions,Methane_Emissions,NitrousOxide_Emissions,FluorinatedGases_Emissions,Total_emissions,...,HottestMonth,HottestMonthTemperature,YearlyTemperatureChange,CO2_Emissions_Bucket,GreenhouseGas_Emissions_Bucket,Methane_Emissions_Bucket,NitrousOxide_Emissions_Bucket,FluorinatedGases_Emissions_Bucket,Total_emissions_Bucket,YearlyTemperatureChange_Bucket
0,1,Africa,1970,Million metric tons of CO2 equivalent,64870.726312,184.519702,165907.310511,83831.162105,17020.902415,331814.621045,...,May,25.910,0.000000,"(63267.757, 3408222.516]","(75.487, 233.043]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]","(-0.0617, 0.0133]"
1,2,Africa,1971,Million metric tons of CO2 equivalent,65698.449942,230.533470,167785.877827,84608.622741,17248.271662,335571.755642,...,May,25.542,-0.388500,"(63267.757, 3408222.516]","(75.487, 233.043]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]","(-0.581, -0.339]"
2,3,Africa,1972,Million metric tons of CO2 equivalent,71123.001404,256.200155,176209.783378,87357.595343,17472.986474,352419.566753,...,September,26.230,0.266000,"(63267.757, 3408222.516]","(233.043, 979.53]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]","(0.237, 0.382]"
3,4,Africa,1973,Million metric tons of CO2 equivalent,80463.562871,292.605500,189116.514777,90787.750991,17572.595426,378233.029565,...,May,26.209,0.201000,"(63267.757, 3408222.516]","(233.043, 979.53]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]","(0.113, 0.237]"
4,5,Africa,1974,Million metric tons of CO2 equivalent,84212.720530,328.357094,193014.219266,90651.556696,17821.584994,386028.438580,...,June,25.670,-0.569667,"(63267.757, 3408222.516]","(233.043, 979.53]","(83782.85, 4354604.956]","(16267.388, 680577.362]","(4601.992, 162808.507]","(167565.7, 8709209.912]","(-0.581, -0.339]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265227,265228,Zimbabwe,2009,Million metric tons of CO2 equivalent,2385.117265,43.824745,4726.987794,1673.450995,624.594788,9453.975589,...,December,24.731,-0.168333,"(1993.736, 4133.465]","(24.897, 75.487]","(4327.655, 7638.787]","(1221.238, 1953.222]","(596.912, 883.45]","(8655.31, 15277.575]","(-0.178, -0.0617]"
265228,265229,Zimbabwe,2010,Million metric tons of CO2 equivalent,2738.667402,43.529480,5218.071099,1753.776572,682.097645,10436.142198,...,October,25.943,0.609000,"(1993.736, 4133.465]","(24.897, 75.487]","(4327.655, 7638.787]","(1221.238, 1953.222]","(596.912, 883.45]","(8655.31, 15277.575]","(0.382, 0.619]"
265229,265230,Zimbabwe,2011,Million metric tons of CO2 equivalent,3090.699304,43.806655,5651.261942,1806.427599,710.328385,11302.523885,...,November,25.521,-0.383833,"(1993.736, 4133.465]","(24.897, 75.487]","(4327.655, 7638.787]","(1221.238, 1953.222]","(596.912, 883.45]","(8655.31, 15277.575]","(-0.581, -0.339]"
265230,265231,Zimbabwe,2012,Million metric tons of CO2 equivalent,3188.778694,44.685379,5800.089127,1842.329773,724.295280,11600.178253,...,November,24.606,-0.081083,"(1993.736, 4133.465]","(24.897, 75.487]","(4327.655, 7638.787]","(1221.238, 1953.222]","(596.912, 883.45]","(8655.31, 15277.575]","(-0.178, -0.0617]"


In [22]:
# keep three decimal places for temperature
merged_df['AverageYearlyTemperature']=merged_df['AverageYearlyTemperature'].round(3)

merged_df['ColdestMonthTemperature']=merged_df['ColdestMonthTemperature'].round(3)

merged_df['HottestMonthTemperature']=merged_df['HottestMonthTemperature'].round(3)

# Convert emissions + yearly temp change columns to float with maximum 3 decimal places
cols = ['CO2_Emissions', 'GreenhouseGas_Emissions', 'Methane_Emissions', 
                  'NitrousOxide_Emissions', 'FluorinatedGases_Emissions', 'Total_emissions', 'YearlyTemperatureChange']
merged_df[cols] = merged_df[cols].round(3)


In [23]:
# Loading integrated/final dataset.
merged_df.to_csv('Stagged_data.csv')

In [27]:
### SETTING UP DBMS

import sqlite3

# Load the CSV file into a DataFrame
#merged_df = pd.read_csv('Stagged_data.csv')

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('datamart.db')

# Create a cursor object
cursor = conn.cursor()

# Drop bucket columns
bucket_cols = ['CO2_Emissions_Bucket', 'GreenhouseGas_Emissions_Bucket', 
               'Methane_Emissions_Bucket', 'NitrousOxide_Emissions_Bucket', 
               'FluorinatedGases_Emissions_Bucket', 'Total_emissions_Bucket', 
               'YearlyTemperatureChange_Bucket']
existing_bucket_cols = [col for col in bucket_cols if col in merged_df.columns]

# Drop bucket columns if they exist
if existing_bucket_cols:
    merged_df.drop(columns=existing_bucket_cols, inplace=True)

# Define SQL statement to create a table
create_table_sql = """
CREATE TABLE IF NOT EXISTS Emissions (
    SurrogateKeys INTEGER PRIMARY KEY,
    Country TEXT,
    Year INTEGER,
    EmissionsUnit TEXT,
    CO2_Emissions REAL,
    GreenhouseGas_Emissions REAL,
    Methane_Emissions REAL,
    NitrousOxide_Emissions REAL,
    FluorinatedGases_Emissions REAL,
    Total_emissions REAL
);
"""

# Execute SQL statement to create the table
cursor.execute(create_table_sql)

# Insert DataFrame data into the database
merged_df.to_sql('Emissions', conn, if_exists='replace', index=False)

#Create dimension tables
create_table_sql = """
CREATE TABLE IF NOT EXISTS Country_Dimension (
    Country_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Country TEXT NOT NULL
);
"""
cursor.execute(create_table_sql)

create_table_sql = """
CREATE TABLE IF NOT EXISTS Time_Dimension (
    Time_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Year INT NOT NULL
);
"""
cursor.execute(create_table_sql)

create_table_sql = """
CREATE TABLE IF NOT EXISTS Temperature_Dimension (
    Temperature_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Coldest_Month_Temperature FLOAT,
    Hottest_Month_Temperature FLOAT,
    Average_Yearly_Temperature FLOAT,
    Yearly_Temperature_Change FLOAT
);
"""
cursor.execute(create_table_sql)

create_table_sql = """
CREATE TABLE IF NOT EXISTS Emissions_Dimension (
    Emissions_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    CO2_Emissions FLOAT,
    GreenhouseGas_Emissions FLOAT,
    Methane_Emissions FLOAT,
    NitrousOxide_Emissions FLOAT,
    FluorinatedGases_Emissions FLOAT,
    Total_Emissions FLOAT
);
"""
cursor.execute(create_table_sql)

create_table_sql = """
CREATE TABLE IF NOT EXISTS Month_Dimension (
    Month_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Hottest_Month TEXT,
    Coldest_Month TEXT
);
"""
cursor.execute(create_table_sql)

# Populate Country Dimension Table
countries = merged_df['Country'].unique()
for country in countries:
    cursor.execute("INSERT INTO Country_Dimension (Country) VALUES (?)", (country,))
conn.commit()


# Populate Time Dimension Table
years = merged_df['Year'].unique()

for year in years:
    year_int = int(year)
    cursor.execute("INSERT INTO Time_Dimension (Year) VALUES (?)", (year_int,))


conn.commit()


# Populate Temperature Dimension Table
temps = merged_df[['ColdestMonthTemperature', 'HottestMonthTemperature', 
                   'AverageYearlyTemperature', 'YearlyTemperatureChange']].drop_duplicates()
for index, row in temps.iterrows():
    cursor.execute("INSERT INTO Temperature_Dimension (Coldest_Month_Temperature, Hottest_Month_Temperature, Average_Yearly_Temperature, Yearly_Temperature_Change) VALUES (?, ?, ?, ?)",
                   (row['ColdestMonthTemperature'], row['HottestMonthTemperature'],
                    row['AverageYearlyTemperature'], row['YearlyTemperatureChange']))
conn.commit()

# Populate Emissions Dimension Table
emissions = merged_df[['CO2_Emissions', 'GreenhouseGas_Emissions', 'Methane_Emissions', 
                       'NitrousOxide_Emissions', 'FluorinatedGases_Emissions', 'Total_emissions']].drop_duplicates()
for index, row in emissions.iterrows():
    cursor.execute("INSERT INTO Emissions_Dimension (CO2_Emissions, GreenhouseGas_Emissions, Methane_Emissions, NitrousOxide_Emissions, FluorinatedGases_Emissions, Total_Emissions) VALUES (?, ?, ?, ?, ?, ?)",
                   (row['CO2_Emissions'], row['GreenhouseGas_Emissions'], row['Methane_Emissions'],
                    row['NitrousOxide_Emissions'], row['FluorinatedGases_Emissions'], row['Total_emissions']))
conn.commit()

# Populate Month Dimension Table
months = merged_df[['ColdestMonth', 'HottestMonth']].drop_duplicates()
for index, row in months.iterrows():
    cursor.execute("INSERT INTO Month_Dimension (Coldest_Month, Hottest_Month) VALUES (?, ?)",
                   (row['ColdestMonth'], row['HottestMonth']))
conn.commit()





In [28]:
## Create Emissions fact table

# fact table schema
cursor.execute('''CREATE TABLE IF NOT EXISTS Fact_Emissions (
                    ID INTEGER PRIMARY KEY,
                    CountryID INTEGER,
                    TimeID INTEGER,
                    TemperatureID INTEGER,
                    EmissionsID INTEGER,
                    MonthID INTEGER,
                    FOREIGN KEY (CountryID) REFERENCES Country_Dimension (ID),
                    FOREIGN KEY (TimeID) REFERENCES Time_Dimension (ID),
                    FOREIGN KEY (TemperatureID) REFERENCES Temperature_Dimension (ID),
                    FOREIGN KEY (EmissionsID) REFERENCES Emissions_Dimension (ID),
                    FOREIGN KEY (MonthID) REFERENCES Month_Dimension (ID)
                )''')

# Populate the fact table
for index, row in merged_df.iterrows():
    # Get the dimension IDs for each attribute
    country_result = cursor.execute("SELECT Country_ID FROM Country_Dimension WHERE Country = ?", (row['Country'],)).fetchone()
    country_id = country_result[0] if country_result is not None else None
    
    time_result = cursor.execute("SELECT Time_ID FROM Time_Dimension WHERE Year = ?", (row['Year'],)).fetchone()
    time_id = time_result[0] if time_result is not None else None
    
    temperature_result = cursor.execute("SELECT Temperature_ID FROM Temperature_Dimension WHERE Coldest_Month_Temperature = ? AND Hottest_Month_Temperature = ? AND Average_Yearly_Temperature = ? AND Yearly_Temperature_Change = ?",
                                        (row['ColdestMonthTemperature'], row['HottestMonthTemperature'], row['AverageYearlyTemperature'], row['YearlyTemperatureChange'])).fetchone()
    temperature_id = temperature_result[0] if temperature_result is not None else None
    
    emissions_result = cursor.execute("SELECT Emissions_ID FROM Emissions_Dimension WHERE CO2_Emissions = ? AND GreenhouseGas_Emissions = ? AND Methane_Emissions = ? AND NitrousOxide_Emissions = ? AND FluorinatedGases_Emissions = ? AND Total_Emissions = ?",
                                      (row['CO2_Emissions'], row['GreenhouseGas_Emissions'], row['Methane_Emissions'], row['NitrousOxide_Emissions'], row['FluorinatedGases_Emissions'], row['Total_emissions'])).fetchone()
    emissions_id = emissions_result[0] if emissions_result is not None else None
    
    month_result = cursor.execute("SELECT Month_ID FROM Month_Dimension WHERE Coldest_Month = ? AND Hottest_Month = ?", (row['ColdestMonth'], row['HottestMonth'])).fetchone()
    month_id = emissions_result[0] if month_result is not None else None
    
    # Insert the data into the fact table
    cursor.execute("INSERT INTO Fact_Emissions (CountryID, TimeID, TemperatureID, EmissionsID, MonthID) VALUES (?, ?, ?, ?, ?)",
                   (country_id, time_id, temperature_id, emissions_id, month_id))

conn.commit()
conn.close()